# Reduce 90% by 2040

*Goal*: Can we calculate what percentage we would have to reduce the January emissions proposal to reduce emissions from large buildings by 90% by 2040?



In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [16]:
import os
import shutil

In [2]:
from baseline_model import BaselineBEPSModel

In [20]:
JAN_TIMELINE_PATH = '../data/input_data/scen_1_reformatted.csv'
BUILDING_DATA_PATH = '../data/input_data/building_data_recleaned_8_3.csv'

FINES_YEARS = [2030, 2035, 2040, 2045, 2050]
FINE_PER_SQFT = 2.5

EMISSIONS_PATH = '../data/input_data/energy_emissions.csv'

In [4]:
jan_timeline = pd.read_csv(JAN_TIMELINE_PATH)
jan_timeline.head()

,Unnamed: 0,year,building_type,sq_ft,sq_ft_classification,ghgi,max_size,min_size
0,0,2027,College/University,>220K Buildings,A,2.83,1000000,220000
1,1,2027,College/University,>90-220K Buildings,B,2.83,220000,90000
2,2,2027,College/University,>50-90K Buildings,C,NaN,90000,50000
3,3,2027,College/University,>30-50K Buildings,D,NaN,50000,30000
4,4,2027,College/University,>20-30K Buildings,E,NaN,50000,20000


In [9]:
JAN_BASELINE_MODEL_PATH = 'jan_baseline_8_3.csv'
jan_baseline = pd.read_csv(JAN_BASELINE_MODEL_PATH)

# emissions in 2026 will be the same as emissions in 2027 w/o improvements to infrastructure
JAN_BASELINE_2026_EMISSIONS_KG = jan_baseline[jan_baseline['year'] == 2027]['expected_baseline'].sum()
JAN_BASELINE_2026_EMISSIONS_KG

459452209.8585

In [11]:
# calculate 90% reduction level for 2040

TARGET_FOR_2040_KG = JAN_BASELINE_2026_EMISSIONS_KG * 0.1
TARGET_FOR_2040_KG

45945220.98585001

In [13]:
# method to create temporary file for emissions
# percent is % of original baseline, not percent reduction of original baseline
def create_temp_emissions_timeline_file(percent):
    timeline = pd.read_csv(JAN_TIMELINE_PATH)
    timeline['ghgi'] = timeline['ghgi'] * (percent/100.0)
    file_name = f'tmp/timeline_{percent}_percent_of_policy.csv'
    timeline.to_csv(file_name)
    return file_name

In [29]:
def find_reduction_percent(target_kg, target_year, timeline_path, emissions_path, building_data_path, fine_years, fine_per_sqft):
    # make temporary directory for timeline files
    os.mkdir('tmp')

    min_emissions = target_kg - 5000
    max_emissions = target_kg + 5000

    high = 100
    low = 0
    found_target = False

    while not found_target:
        percent_of_orig_emissions = (high + low) / 2
        reduced_timeline_path = create_temp_emissions_timeline_file(percent_of_orig_emissions)

        model = BaselineBEPSModel(emissions_path, reduced_timeline_path, building_data_path, fine_years, fine_per_sqft)
        model.calculate_baseline_model(target_year, target_year)
        emissions_in_2040 = model.scenario_results['compliant_emissions'].sum()

        if min_emissions < emissions_in_2040 < max_emissions:
            print(f'You can achieve {target_kg} kg/yr in {target_year} by reducing the baselines by {100 - percent_of_orig_emissions}%.')
            # move file
            os.rename(reduced_timeline_path, reduced_timeline_path.replace('tmp/', ''))
                      
            # turn model into csv
            model.scenario_results.to_csv(f'emissions_{target_year}_{percent_of_orig_emissions}_of_policy.csv')

            # delete tmp directory
            shutil.rmtree('tmp')

            found_target = True

        else:
            if emissions_in_2040 < target_kg:
                low = percent_of_orig_emissions
            else:
                high = percent_of_orig_emissions

In [30]:
find_reduction_percent(TARGET_FOR_2040_KG, 2040, JAN_TIMELINE_PATH, EMISSIONS_PATH, BUILDING_DATA_PATH, FINES_YEARS, FINE_PER_SQFT)

/Users/ihc/programming/350_seattle/BPS/experiments/benchmarking_models/models/baseline_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['year'] = pd.Series([year]*len(temp_df))
/Users/ihc/programming/350_seattle/BPS/experiments/benchmarking_models/models/baseline_model.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['expected_baseline'] = input_df.apply(lambda building: self._get_expected_baseline(building, year), axis=1)


Model calculations complete. Access the model dataframe as model_name.scenario_results


/Users/ihc/programming/350_seattle/BPS/experiments/benchmarking_models/models/baseline_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['year'] = pd.Series([year]*len(temp_df))
/Users/ihc/programming/350_seattle/BPS/experiments/benchmarking_models/models/baseline_model.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['expected_baseline'] = input_df.apply(lambda building: self._get_expected_baseline(building, year), axis=1)


Model calculations complete. Access the model dataframe as model_name.scenario_results


/Users/ihc/programming/350_seattle/BPS/experiments/benchmarking_models/models/baseline_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['year'] = pd.Series([year]*len(temp_df))
/Users/ihc/programming/350_seattle/BPS/experiments/benchmarking_models/models/baseline_model.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['expected_baseline'] = input_df.apply(lambda building: self._get_expected_baseline(building, year), axis=1)


...

Model calculations complete. Access the model dataframe as model_name.scenario_results


/Users/ihc/programming/350_seattle/BPS/experiments/benchmarking_models/models/baseline_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['year'] = pd.Series([year]*len(temp_df))
/Users/ihc/programming/350_seattle/BPS/experiments/benchmarking_models/models/baseline_model.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['expected_baseline'] = input_df.apply(lambda building: self._get_expected_baseline(building, year), axis=1)


Model calculations complete. Access the model dataframe as model_name.scenario_results


/Users/ihc/programming/350_seattle/BPS/experiments/benchmarking_models/models/baseline_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['year'] = pd.Series([year]*len(temp_df))
/Users/ihc/programming/350_seattle/BPS/experiments/benchmarking_models/models/baseline_model.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['expected_baseline'] = input_df.apply(lambda building: self._get_expected_baseline(building, year), axis=1)


Model calculations complete. Access the model dataframe as model_name.scenario_results
You can achieve 45945220.98585001 kg/yr in 2040 by reducing the baselines by 68.2373046875%.
